# Imports

In [1]:
from pyiron_contrib.tinybase.node import AbstractNode, FunctionNode, SeriesNode, LoopNode

/home/ponder/science/phd/dev/pyiron_contrib/pyiron_contrib/__init__.py:9: UserWarning: pyiron module not found, importing Project from pyiron_base
  warnings.warn("pyiron module not found, importing Project from pyiron_base")


In [2]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [3]:
import logging
logging.getLogger().setLevel(20)

In [4]:
import numpy as np

# Function Node

## Basic

In [5]:
def calc_fib(n):
    import time
    n1 = n2 = 1
    for i in range(n):
        time.sleep(.1)
        x = n1 + n2
        n1 = n2
        n2 = x
    return x

In [6]:
f = FunctionNode(calc_fib)

In [7]:
f.input.storage

DataContainer([])

In [8]:
f.input.args

[]

In [9]:
f.input.kwargs

{}

In [10]:
f.input.kwargs['n'] = 10

In [11]:
f.input.kwargs

{'n': 10}

In [12]:
exe = f.run()

In [13]:
exe._run_machine._data

{'status': (ReturnStatus(Code.DONE, None),),
 'output': (<pyiron_contrib.tinybase.node.FunctionOutput at 0x7fa49a99ec90>,)}

In [14]:
exe.output[0].result

144

In [15]:
exe._run_time, exe._collect_time

(1.0009997060114983, 9.59599856287241e-06)

## We don't actually have to use an executor if we just want a result

In [16]:
f = FunctionNode(calc_fib)

In [17]:
f.input.kwargs['n'] = 10

In [18]:
f.execute()

(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.node.FunctionOutput at 0x7fa49a76d510>)

## Do the same but in the background

In [19]:
f = FunctionNode(calc_fib)

In [20]:
f.input.kwargs['n'] = 100

In [21]:
exe = f.run(how='background')

In [22]:
exe._run_machine._data

{}

In [23]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [24]:
exe.wait()

In [25]:
exe.output[0].result

927372692193078999176

## Do the same but in the background as process

In [26]:
fib_node = FunctionNode(calc_fib)

In [27]:
fib_node.input.kwargs['n'] = 100

In [28]:
exe = fib_node.run(how='process')

In [29]:
exe._run_machine._data

{}

In [30]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [31]:
exe.wait()

In [32]:
exe.output[0].result

927372692193078999176

# Executors handle single nodes and lists of them on the same footing

In [33]:
nodes = [FunctionNode(calc_fib) for _ in range(10)]

In [34]:
for i, n in enumerate(nodes):
    n.input.kwargs['n'] = 3 + i

## With the basic executor

In [35]:
exe = Executor(nodes)
exe.run()

In [36]:
exe.output

(<pyiron_contrib.tinybase.node.FunctionOutput at 0x7fa49acb3750>,
 <pyiron_contrib.tinybase.node.FunctionOutput at 0x7fa49a78b350>)

In [37]:
exe.output[5].result

55

## With the process executor

In [38]:
exe = ProcessExecutor(nodes)
exe.run()

In [39]:
exe.wait()

In [40]:
exe.status

[ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None)]

In [41]:
exe.output

In [42]:
exe.output[5].result

55

# SeriesNode

In [43]:
s = SeriesNode()

In [44]:
f1 = FunctionNode(calc_fib)

In [45]:
f2 = FunctionNode(np.sqrt)

In [46]:
def transfer(input, output):
    input.args = [output.result]

In [47]:
s.input.first(f1).then(f2, transfer)

In [48]:
s.input.nodes[0].input.kwargs['n'] = 10

In [49]:
status, output = s.execute()

In [50]:
status

ReturnStatus(Code.DONE, None)

In [51]:
output.result

12.0

# Loop Node

## Simple repeat loop

In [52]:
l = LoopNode()

In [53]:
l.input.node = FunctionNode(lambda: np.random.rand())

In [54]:
l.input.repeat(10, restart=lambda output, input, scratch: print(output.result))

In [55]:
exe = l.run()

0.6416842803528255
0.7583838349829116
0.25412630351718535
0.978146926974964
0.6346764217817196
0.8760354316006344
0.3598792613379673
0.7532969335152777
0.3682944984993325


In [56]:
exe.status

(ReturnStatus(Code.DONE, None),)

## Loop with a termination condition

In [57]:
l = LoopNode()

In [58]:
l.input.node = FunctionNode(lambda: np.random.rand())

In [59]:
l.input.control_with(
    condition=lambda node, output, scratch: output.result < .05,
    restart=lambda output, input, scratch: print(output.result)
)

In [60]:
exe = l.run()

0.9413063812991156
0.057607401981599304
0.5397672296450867
0.09250834651031281
0.07879485371081929
0.11682404864093698
0.40972529768552957
0.4949451538792201
0.4185353179411203
0.8901597039186674
0.7351951144010666
0.23869310822669132
0.31807750515467925
0.4612495341559427
0.7640290216688888
0.3869288752008373
0.23391774163562495
0.5139434388248725
0.6190472000582091
0.22725085086512875
0.8568498193403175
0.20466606514335917
0.6031013104131141
0.4322284299797473
0.6921598384356622
0.9143146262866392
0.7142585471813697
0.8067120253887571
0.2985613290985082
0.08197401100526269
0.8502914897450804
0.5441851005084206
0.4514940877891841
0.4895956404459735
0.23293653173840512
0.9152221474508541
0.35937794575487036
0.9560404151460253
0.1681320614012014
0.3887573415069143
0.936771887470757
0.8987739992014511
0.22822592639583683
0.5306744820161604
0.21711390374139883
0.7720820846158695
0.694832684749003
0.9880804928246634
0.8877319250523327
0.7389376534050393


In [61]:
exe.status[0]

ReturnStatus(Code.DONE, None)

In [62]:
exe.output[0].result

0.03279942765741395